In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense

from keras import backend as K
K.set_image_dim_ordering('th')

from mydatools import img_plot

%matplotlib inline

Using Theano backend.
Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)


In [2]:
# path = 'data/input/all/'
path = 'data/input/sample/'

In [3]:
def get_batches(dirname, gen=ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, target_size=target_size)
    X = np.concatenate([batches.next()[0] for i in range(batches.samples)])
    y = np.concatenate([batches.next()[1] for i in range(batches.samples)])
    return X, y, batches

def get_class(a_onehot):
    return a_onehot.argmax(axis=1)

## VGG16 finetune

In [4]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.samples, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.samples)

In [5]:
tra_batches = get_batches(path+'train', shuffle=True, batch_size=64)
val_batches = get_batches(path+'val', shuffle=False, batch_size=64)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [6]:
# vgg16
from vgg16 import Vgg16
model = Vgg16()

In [7]:
model.finetune(tra_batches)

#Not sure if we set this for all fits
model.model.optimizer.lr = 0.01

In [8]:
model.fit(tra_batches, val_batches, nb_epoch=3)

Epoch 1/3
28/28 [==============================] - 39s - loss: 0.7233 - acc: 0.9202 - val_loss: 0.2740 - val_acc: 0.9740
Epoch 2/3
28/28 [==============================] - 77s - loss: 0.2236 - acc: 0.9827 - val_loss: 0.3508 - val_acc: 0.9706
Epoch 3/3
28/28 [==============================] - 81s - loss: 0.2393 - acc: 0.9788 - val_loss: 0.2927 - val_acc: 0.9706
